In [1]:
!pip install pennylane-lightning-gpu
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 

In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer,load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import torch
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
# ------------------------------
# 1. Load and preprocess the data
# ------------------------------
# Load the UCI Breast Cancer (WDBC) dataset
data = load_breast_cancer()
X = data.data   # 30 features
y = data.target # 0 for malignant, 1 for benign

X_Scaled=StandardScaler().fit_transform(X)
df=pd.DataFrame(X_Scaled,columns=data.feature_names)
# Reduce to 4 features using PCA (to keep the quantum circuit manageable)
pca = PCA(n_components=8)
X_reduced = pca.fit_transform(X_Scaled)

print(pca.explained_variance_ratio_.sum())

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced,y, test_size=0.2, random_state=42)
print(torch.cuda.is_available())
print("Size of Train_set : ",X_train.shape)
print("Size of Test_set : ",X_test.shape)

0.925982538696557
True
Size of Train_set :  (455, 8)
Size of Test_set :  (114, 8)


In [3]:
import torch

print("Total CUDA GPUs PyTorch can see:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

Total CUDA GPUs PyTorch can see: 2
Device 0: Tesla T4
Device 1: Tesla T4


In [4]:
svm = SVC(kernel='linear')
st=time.time()
svm.fit(X_train, y_train)
et=time.time()
y_pred_svm = svm.predict(X_test)
accuracy_svm = np.mean(y_pred_svm == y_test) * 100
print(f"Classical Kernel SVM Accuracy: {accuracy_svm:.2f}%")
print(f"Total Time Taken by Classical Kernel:{et-st}")
print(torch.device)

Classical Kernel SVM Accuracy: 98.25%
Total Time Taken by Classical Kernel:0.0625147819519043
<class 'torch.device'>


In [5]:
# # Multiprocessing using pennylane whereby using multiple gpus per process 
# import pennylane as qml
# import multiprocessing as mp
# import numpy as np
# import os
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# NUM_QUBITS = 7
# NUM_GPUS = 2
# NUM_LAYERS = 2

# def make_device(gpu_id):
#     os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
#     return qml.device("lightning.gpu", wires=NUM_QUBITS)

# def quantum_kernel_element(x1, x2, dev, weights):
#     @qml.qnode(dev)
#     def circuit(weights):  # <- pass weights as an argument
#         qaoa_feature_map(x1, weights)
#         qml.adjoint(qaoa_feature_map)(x2, weights)
#         return qml.probs(wires=range(NUM_QUBITS))
    
#     return circuit(weights)[0].item()

# def compute_rows(start_idx, end_idx, X1, X2, gpu_id, return_dict, weights):
#     dev = make_device(gpu_id)
#     partial_kernel = np.zeros((end_idx - start_idx, len(X2)))
#     for i_local, i in enumerate(range(start_idx, end_idx)):
#         for j in range(len(X2)):
#             partial_kernel[i_local, j] = quantum_kernel_element(X1[i], X2[j], dev, weights)
#     return_dict[gpu_id] = (start_idx, end_idx, partial_kernel)

# def compute_kernel_matrix(X1, X2, weights, num_gpus=2):
#     n = len(X1)
#     chunk_size = n // num_gpus
#     manager = mp.Manager()
#     return_dict = manager.dict()
#     processes = []

#     mp.set_start_method('spawn', force=True)

#     for gpu_id in range(num_gpus):
#         start = gpu_id * chunk_size
#         end = n if gpu_id == num_gpus - 1 else (gpu_id + 1) * chunk_size
#         p = mp.Process(target=compute_rows, args=(start, end, X1, X2, gpu_id, return_dict, weights))
#         p.start()
#         processes.append(p)

#     for p in processes:
#         p.join()

#     # Assemble final kernel matrix (X1 × X2)
#     K = np.zeros((n, len(X2)))
#     for gpu_id in return_dict.keys():
#         start, end, chunk = return_dict[gpu_id]
#         K[start:end, :] = chunk
#     return K

In [6]:
# opt = qml.GradientDescentOptimizer(stepsize=0.1)
# weights = np.random.uniform(low=0, high=2*np.pi, size=(NUM_LAYERS, NUM_QUBITS))  # shape for QAOAEmbedding

# for i in range(30):
#     weights = opt.step(cost, weights)
#     acc = -cost(weights)
#     print(f"Step {i+1}: Training accuracy = {acc:.4f}")

In [7]:
import pennylane as qml
import multiprocessing as mp
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

NUM_QUBITS = 8

# Define feature map options
def angle_feature_map(x, wires):
    """Angle Embedding feature map with additional rotations"""
    x = np.pi * (x - min(x)) / (max(x) - min(x) + 1e-6)
    qml.AngleEmbedding(x, wires=wires, rotation='Z')
    for i in range(NUM_QUBITS):
        qml.RY(x[i], wires=i)

def amplitude_feature_map(x, wires):
    """Amplitude Embedding feature map with padding and normalization"""
    x = x / np.sqrt(np.sum(np.abs(x)**2) + 1e-6)
    qml.AmplitudeEmbedding(x, wires=wires, pad_with=0.0, normalize=True)
    for i in range(NUM_QUBITS):
        qml.RZ(weights[i], wires=i)

FEATURE_MAPS = {
    'angle': angle_feature_map,
    'amplitude': amplitude_feature_map
}

def make_device():
    """Create a single GPU device"""
    return qml.device("lightning.gpu", wires=NUM_QUBITS)

def quantum_kernel_element(x1, x2, dev, weights, feature_map_type='angle'):
    feature_map = FEATURE_MAPS[feature_map_type]
    
    @qml.qnode(dev)
    def circuit(weights):
        feature_map(x1, wires=range(NUM_QUBITS))
        qml.adjoint(feature_map)(x2, wires=range(NUM_QUBITS))
        return qml.probs(wires=range(NUM_QUBITS))
    
    return circuit(weights)[0].item()

def compute_kernel_matrix(X1, X2, weights, feature_map_type='angle'):
    """Compute kernel matrix using a single GPU"""
    dev = make_device()
    n1, n2 = len(X1), len(X2)
    K = np.zeros((n1, n2))
    
    for i in range(n1):
        for j in range(n2):
            K[i, j] = quantum_kernel_element(X1[i], X2[j], dev, weights, feature_map_type)
    
    return K


feature_map_type = 'angle'
weights = np.random.uniform(low=0, high=2*np.pi, size=(NUM_QUBITS,))

print("Computing training kernel matrix...")
K_train = compute_kernel_matrix(X_train, X_train, weights, feature_map_type)
print("Computing test kernel matrix...")
K_test = compute_kernel_matrix(X_test, X_train, weights, feature_map_type)

# clf = SVC(kernel='precomputed', C=1.0, class_weight='balanced')
clf = SVC(kernel='precomputed', C=1.0, class_weight='balanced')
clf.fit(K_train, y_train)

train_acc = accuracy_score(y_train, clf.predict(K_train))
test_acc = accuracy_score(y_test, clf.predict(K_test))
    
print(f"Training accuracy with {feature_map_type} feature map: {train_acc:.4f}")
print(f"Test accuracy with {feature_map_type} feature map: {test_acc:.4f}")

Computing training kernel matrix...
Computing test kernel matrix...
Training accuracy with angle feature map: 0.9912
Test accuracy with angle feature map: 0.9561


In [8]:
clf = SVC(kernel='precomputed')
clf.fit(K_train, y_train)

train_acc = accuracy_score(y_train, clf.predict(K_train))
test_acc = accuracy_score(y_test, clf.predict(K_test))
    
print(f"Training accuracy with {feature_map_type} feature map: {train_acc:.4f}")
print(f"Test accuracy with {feature_map_type} feature map: {test_acc:.4f}")

Training accuracy with angle feature map: 0.9912
Test accuracy with angle feature map: 0.9561


In [9]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, K_train, y_train, cv=5)
print(f"Cross-validation scores: {scores.mean():.4f} ± {scores.std():.4f}")

Cross-validation scores: 0.9582 ± 0.0264


In [10]:
clf_classical = SVC(kernel='linear', C=1.0, class_weight='balanced')
clf_classical.fit(X_train, y_train)
print(f"Classical RBF Test Accuracy: {clf_classical.score(X_test, y_test):.4f}")

Classical RBF Test Accuracy: 0.9649


In [11]:
import pennylane as qml
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

NUM_QUBITS = 4  # Reduced for speed

# Define feature maps (weights removed)
def angle_feature_map(x, wires):
    qml.AngleEmbedding(x, wires=wires, rotation='Z')

def amplitude_feature_map(x, wires):
    x = x / np.sqrt(np.sum(np.abs(x)**2) + 1e-6)
    qml.AmplitudeEmbedding(x, wires=wires, pad_with=0.0, normalize=True)

FEATURE_MAPS = {
    'angle': angle_feature_map,
    'amplitude': amplitude_feature_map
}

def make_device():
    return qml.device("lightning.gpu", wires=NUM_QUBITS)  # CPU for simplicity

def quantum_kernel_element(x1, x2, dev, feature_map_type='angle'):
    feature_map = FEATURE_MAPS[feature_map_type]
    
    @qml.qnode(dev)
    def circuit():
        feature_map(x1, wires=range(NUM_QUBITS))
        qml.adjoint(lambda wires: feature_map(x2, wires))(wires=range(NUM_QUBITS))
        return qml.probs(wires=range(NUM_QUBITS))
    
    return circuit()[0].item()

def compute_kernel_matrix(X1, X2, feature_map_type='angle'):
    dev = make_device()
    n1, n2 = len(X1), len(X2)
    K = np.zeros((n1, n2))
    for i in range(n1):
        for j in range(n2):
            K[i, j] = quantum_kernel_element(X1[i], X2[j], dev, feature_map_type)
    return K

def main():
    data = load_breast_cancer()
    X = data.data
    y = data.target
    X_scaled = StandardScaler().fit_transform(X)
    pca = PCA(n_components=NUM_QUBITS)
    X_reduced = pca.fit_transform(X_scaled)

    X_train, X_test, y_train, y_test = train_test_split(
        X_reduced, y, test_size=0.2, random_state=42
    )

    feature_map_type = 'amplitude'  # Faster with simpler map

    print("Computing training kernel matrix...")
    K_train = compute_kernel_matrix(X_train, X_train, feature_map_type)
    print("Computing test kernel matrix...")
    K_test = compute_kernel_matrix(X_test, X_train, feature_map_type)

    clf = SVC(kernel='precomputed', C=1.0, class_weight='balanced')
    clf.fit(K_train, y_train)

    train_acc = accuracy_score(y_train, clf.predict(K_train))
    test_acc = accuracy_score(y_test, clf.predict(K_test))
    
    print(f"Training accuracy with {feature_map_type}: {train_acc:.4f}")
    print(f"Test accuracy with {feature_map_type}: {test_acc:.4f}")

if __name__ == '__main__':
    main()

Computing training kernel matrix...
Computing test kernel matrix...
Training accuracy with amplitude: 0.6857
Test accuracy with amplitude: 0.6053
